<details><summary> </summary>

# Skip notebook test

</details>

In [1]:
import cudf
from cuopt import routing
import numpy as np
import os
from cuopt.routing import utils
from scipy.spatial import distance

# Benchmark Gehring & Homberger
## Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)

While other notebooks such as [cvrptw_service_team_routing.ipynb](cvrptw_service_team_routing.ipynb) focus on the cuOpt API and high level problem modeling, here we focus on performance.

cuOpt offers a unique benefit over other solver_settingss, specifically, time to solution.  In addition to achieving world class accuracy, cuOpt also produces these solutions in a time frame that allows for re-optimization in dynamic environments and rapid iteration over possible problem configurations.

Here we are demonstrating this performance on a large popular academic [dataset by Gehing & Homberger](https://www.sintef.no/projectweb/top/vrptw/homberger-benchmark/).  These problems are well studied and used as the basis for comparison for VRP research and product offerings. The particular instance we will test with is from the group of largest (1000 location) problems.  Each problem instance has an associated best known solution, the one we will measure against is shown below

In [2]:
homberger_1000_file = 'notebook_utils/data/C1_10_1.TXT'

best_known_solution = {
    "n_vehicles": 100,
    "cost": 42478.95
}

### Problem Data
The data for this problem instance are provided via text file.  cuOpt has a utility function available specifically for the Gehring & Homberger benchmark which converts the problem into the components required by cuOpt.

In [3]:
orders, vehicle_capacity, n_vehicles = utils.create_from_file(homberger_1000_file)

print("Number of locations          : ", orders["demand"].shape[0]-1)
print("Number of vehicles available : ", n_vehicles)
print("Capacity of each vehicle     : ", vehicle_capacity)
print("\nInitial Orders information")
print(orders)

Number of locations          :  1000
Number of vehicles available :  250
Capacity of each vehicle     :  200

Initial Orders information
      vertex  xcord  ycord  demand  earliest_time  latest_time  service_time  \
0          0  250.0  250.0       0              0         1824             0   
1          1  387.0  297.0      10            200          270            90   
2          2    5.0  297.0      10            955         1017            90   
3          3  355.0  177.0      20            194          245            90   
4          4   78.0  346.0      30            355          403            90   
...      ...    ...    ...     ...            ...          ...           ...   
996      996  330.0  242.0      30            627          671            90   
997      997  332.0  249.0      30             82          144            90   
998      998  375.0   80.0      10            550          598            90   
999      999   94.0  235.0      20            227          266 

### Cost Matrix

In [4]:
coords = list(zip(orders['xcord'].to_arrow().to_pylist(),
                  orders['ycord'].to_arrow().to_pylist()))

cost_matrix = cudf.DataFrame(distance.cdist(coords, coords, 'euclidean')).astype(np.float32)
print(f"Shape of cost matrix: {cost_matrix.shape}")

Shape of cost matrix: (1001, 1001)


### cuOpt DataModel View

Setup the routing.DataModel.

In [5]:
n_locations = len(cost_matrix)

data_model = routing.DataModel(n_locations, n_vehicles)
data_model.add_cost_matrix(cost_matrix)

capacity = cudf.Series([vehicle_capacity] * n_vehicles)
data_model.add_capacity_dimension("demand", orders['demand'], capacity)

data_model.set_order_time_windows(orders['earliest_time'], orders['latest_time'])
data_model.set_order_service_times(orders['service_time'])

### CuOpt SolverSettings

Set up routing.SolverSettings.

In [6]:
solver_settings = routing.SolverSettings()

# set number of climbers that will try to search for an optimal routes in parallel
solver_settings.set_number_of_climbers(2048)

### Solution

Here we will examine the quality of the solution we increase the time budget provided to cuOpt

In [7]:
def solve_problem(data_model, solver_settings, problem_size):
    routing_solution = routing.Solve(data_model, solver_settings)
    if routing_solution.get_status() == 0:
        print("Cost for the routing in time: ", routing_solution.final_cost)
        print("Vehicle count to complete routing: ", routing_solution.vehicle_count)
        utils.show_vehicle_routes(routing_solution.route, ["Depot"]+[str(i) for i in range(1, problem_size+1)])
    else:
        print("NVIDIA cuOpt Failed to find a solution with status : ", routing_solution.get_status())
        
    return(routing_solution.vehicle_count, routing_solution.final_cost)

In [8]:
def solution_eval(vehicles, cost, best_known_solution):
    
    print(f"- cuOpt provides a solution using {vehicles} vehicles")
    print(f"- This represents {vehicles - best_known_solution['n_vehicles']} more than the best known solution")
    print(f"- Vehicle Percent Difference {(vehicles/best_known_solution['n_vehicles'] - 1)*100}% \n\n")
    print(f"- In addition cuOpt provides a solution cost of {cost}") 
    print(f"- Best known solution cost is {best_known_solution['cost']}")
    print(f"- Cost Percent Difference {(cost/best_known_solution['cost'] - 1)*100}%")

**1 Second Time Limit**

In [9]:
solver_settings.set_time_limit(1)
vehicles, cost = solve_problem(data_model, solver_settings, len(cost_matrix))

Cost for the routing in time:  43363.99609375
Vehicle count to complete routing:  100
For vehicle - 0 route is: 

Depot->108->182->470->93->203->716->706->875->693->762->601->232->Depot


For vehicle - 3 route is: 

Depot->544->278->511->308->191->712->898->979->915->732->Depot


For vehicle - 4 route is: 

Depot->615->667->808->926->964->988->277->286->824->355->Depot


For vehicle - 5 route is: 

Depot->6->268->980->210->574->118->897->202->547->Depot


For vehicle - 8 route is: 

Depot->570->350->882->400->752->269->865->75->713->185->Depot


For vehicle - 10 route is: 

Depot->175->144->373->692->133->913->539->891->892->514->Depot


For vehicle - 14 route is: 

Depot->384->853->432->727->288->835->858->361->839->608->994->447->Depot


For vehicle - 18 route is: 

Depot->599->755->704->99->10->649->556->598->273->717->Depot


For vehicle - 19 route is: 

Depot->997->87->585->365->120->521->996->873->576->Depot


For vehicle - 20 route is: 

Depot->931->173->151->137->998->163->26->

In [10]:
# Evaluation:
solution_eval(vehicles, cost, best_known_solution)

- cuOpt provides a solution using 100 vehicles
- This represents 0 more than the best known solution
- Vehicle Percent Difference 0.0% 


- In addition cuOpt provides a solution cost of 43363.99609375
- Best known solution cost is 42478.95
- Cost Percent Difference 2.0834933390538213%


**10 Second Time Limit**

In [11]:
solver_settings.set_time_limit(10)
vehicles, cost = solve_problem(data_model, solver_settings, len(cost_matrix))

Cost for the routing in time:  43090.44921875
Vehicle count to complete routing:  100
For vehicle - 0 route is: 

Depot->164->323->242->419->179->622->450->609->877->753->Depot


For vehicle - 1 route is: 

Depot->380->44->4->17->9->89->18->105->153->811->Depot


For vehicle - 4 route is: 

Depot->488->800->211->359->442->804->715->888->923->355->Depot


For vehicle - 9 route is: 

Depot->76->631->467->333->639->229->730->88->659->110->42->Depot


For vehicle - 10 route is: 

Depot->108->182->470->93->203->716->706->875->693->762->601->232->Depot


For vehicle - 19 route is: 

Depot->394->441->40->282->298->119->610->23->524->Depot


For vehicle - 20 route is: 

Depot->190->842->50->675->822->459->830->872->903->Depot


For vehicle - 25 route is: 

Depot->638->72->573->337->613->947->588->439->751->Depot


For vehicle - 27 route is: 

Depot->285->356->472->393->967->474->688->315->518->Depot


For vehicle - 28 route is: 

Depot->435->250->629->829->523->117->768->657->863->330->248->82

In [12]:
# Evaluation:
solution_eval(vehicles, cost, best_known_solution)

- cuOpt provides a solution using 100 vehicles
- This represents 0 more than the best known solution
- Vehicle Percent Difference 0.0% 


- In addition cuOpt provides a solution cost of 43090.44921875
- Best known solution cost is 42478.95
- Cost Percent Difference 1.43953468423772%


**20 Second Time Limit**

In [13]:
solver_settings.set_time_limit(20)
vehicles, cost = solve_problem(data_model, solver_settings, len(cost_matrix))

Cost for the routing in time:  42792.84765625
Vehicle count to complete routing:  100
For vehicle - 0 route is: 

Depot->164->323->242->419->179->622->450->609->877->753->Depot


For vehicle - 1 route is: 

Depot->380->44->4->17->9->89->18->105->153->811->Depot


For vehicle - 4 route is: 

Depot->488->800->211->359->442->804->715->888->923->355->Depot


For vehicle - 9 route is: 

Depot->76->631->467->333->639->229->730->88->659->110->42->Depot


For vehicle - 10 route is: 

Depot->108->182->470->93->203->716->706->875->693->762->601->232->Depot


For vehicle - 19 route is: 

Depot->394->441->40->282->298->119->610->23->524->Depot


For vehicle - 20 route is: 

Depot->190->842->50->675->822->459->830->872->903->Depot


For vehicle - 25 route is: 

Depot->638->72->573->337->613->947->588->439->751->Depot


For vehicle - 27 route is: 

Depot->285->356->472->393->967->474->688->315->518->Depot


For vehicle - 28 route is: 

Depot->435->250->629->829->523->117->768->657->863->330->248->82

In [14]:
# Evaluation:
solution_eval(vehicles, cost, best_known_solution)

- cuOpt provides a solution using 100 vehicles
- This represents 0 more than the best known solution
- Vehicle Percent Difference 0.0% 


- In addition cuOpt provides a solution cost of 42792.84765625
- Best known solution cost is 42478.95
- Cost Percent Difference 0.7389487175412768%
